In [1]:
#@title
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

df_train = pd.read_csv("drive/My Drive/sougou/data/train2.csv",encoding="UTF-8")
df_train = df_train.drop(["Unnamed: 0"],axis=1).loc[:,["ID","Age","Gender","Education","Query_List","Query_List1","jieba","jieba2"]]
df_train.head()

,ID,Age,Gender,Education,Query_List,Query_List1,jieba,jieba2
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,柔和双沟\t女生\t中财网首页 财经\t网址\t周公解梦大全查询2345\t曹云金再讽郭德纲...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再讽 ...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦..."
2,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 写 范例 三维 绘图 软件 枣 和 酸...
3,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...
4,0D58894CD481D4B3039000661665C0DB,3,2,3,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅 饲育 指南 韩姨 贵阳 古琴 洛基 风速 英美 店主 好吃 重生 儿童 寻家记 锦里 ...,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...


In [3]:
df_train.shape

(88492, 8)

In [4]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

df_test = pd.read_csv("drive/My Drive/sougou/data/test.csv",sep="###__###",header = None,encoding="UTF-8")
df_test.columns = ['ID', 'Query_List']
df_test.head()

,ID,Query_List
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...


In [5]:
# 把网址替换成“网址"
import re
def tihuan_url(query):
  pattern = re.compile(r':*(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]')
  querys = query.split("\t")
  for i in range(len(querys)):
    match = pattern.match(querys[i])
    if match:
      a = querys[i].replace(match.group(),"网址")
      querys[i] = a
  s = "\t"
  query = s.join(querys)
  return query

df_test["Query_List1"] = [tihuan_url(k) for k in df_test["Query_List"]]
df_test.head()

,ID,Query_List,Query_List1
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...


In [0]:
stopwords=pd.read_csv("drive/My Drive/sougou/data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [7]:
import jieba

# 分词但不去停用词
def preprocess(df):
  list0 = []
  for k in df["Query_List1"]:
    list1 = k.split("\t")
    list2 = []
    try:
      for i in list1:
        segs = jieba.lcut(i)
        list2 += segs
      list2 = list(filter(lambda x:len(x)>1, list2)) #没有解析出来的过滤掉
      list2 = list(filter(lambda x:x not in stopwords, list2)) #把停用词过滤掉
      list0.append(" ".join(list2))
    except Exception as e:
        print(list1)
        continue
  return list0

df_test["jieba"] = preprocess(df_test)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.615 seconds.
Prefix dict has been built successfully.


In [8]:
df_test.head()

,ID,Query_List,Query_List1,jieba
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...,陈学冬 作品 刘昊然 谭松韵 211 学校 分数线 味道 好听 假吻 搞笑 电视剧 排行榜 ...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...,0.0052 次方 qq 快速 提现 绝色 倾城 飞烟 马克思主义 基本原理 概论 康世恩 ...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...,黑暗 lpl 夏季 富豪 电玩 英雄 联盟 之电 称王 手机 扫描 手机 二维码 重庆 重钢...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...,中秋 水库 钓鱼 鱼竿 蚯蚓 钓鱼 调漂 传统 鱼钩 鲫鱼汤 做法 大全 鱼饵 直销 戒烟 ...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...,号码 吉凶 退休干部 死后 配偶 郫县 大学 胜利油田 中石化 石油 苏珊 米勒 狮子座 年...


In [0]:
import jieba

# 分词但不去停用词
def preprocess(df):
  list0 = []
  for k in df["Query_List1"]:
    list1 = k.split("\t")
    list2 = []
    try:
      for i in list1:
        segs = jieba.lcut(i)
        list2 += segs
#      list2 = list(filter(lambda x:len(x)>1, list2)) #没有解析出来的过滤掉
#      list2 = list(filter(lambda x:x not in stopwords, list2)) #把停用词过滤掉
      list0.append(" ".join(list2))
    except Exception as e:
        print(list1)
        continue
  return list0

df_test["jieba2"] = preprocess(df_test)

In [10]:
df_test.head()

,ID,Query_List,Query_List1,jieba,jieba2
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...,陈学冬 作品 刘昊然 谭松韵 211 学校 分数线 味道 好听 假吻 搞笑 电视剧 排行榜 ...,陈学冬 将 出 的 作品 刘昊然 与 谭松韵 211 学校 的 分数线 谁 唱 的 味道 好...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...,0.0052 次方 qq 快速 提现 绝色 倾城 飞烟 马克思主义 基本原理 概论 康世恩 ...,e 的 0.0052 次方 qq 怎么 快速 提现 绝色 倾城 飞烟 马克思主义 基本原理 ...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...,黑暗 lpl 夏季 富豪 电玩 英雄 联盟 之电 称王 手机 扫描 手机 二维码 重庆 重钢...,黑暗 文 lpl 夏季 赛 大 富豪 电玩 城 英雄 联盟 之电 竞 称王 手机 怎么 扫描...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...,中秋 水库 钓鱼 鱼竿 蚯蚓 钓鱼 调漂 传统 鱼钩 鲫鱼汤 做法 大全 鱼饵 直销 戒烟 ...,中秋 水库 钓鱼 鱼竿 用 蚯蚓 钓鱼 怎样 调漂 传统 钓 3 号 鱼钩 鲫鱼汤 的 做法...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...,号码 吉凶 退休干部 死后 配偶 郫县 大学 胜利油田 中石化 石油 苏珊 米勒 狮子座 年...,号码 吉凶 退休干部 死后 配偶 郫县 有 哪些 大学 胜利油田 属于 中石化 还是 中 石...


In [11]:
df_test.shape

(100000, 5)

In [12]:
df_all = pd.concat([df_train, df_test]).fillna(0)
df_all.head()

,ID,Age,Gender,Education,Query_List,Query_List1,jieba,jieba2
0,22DD920316420BE2DF8D6EE651BA174B,1.0,1.0,4.0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,柔和双沟\t女生\t中财网首页 财经\t网址\t周公解梦大全查询2345\t曹云金再讽郭德纲...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再讽 ...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再...
1,43CC3AF5A8D6430A3B572337A889AFE4,2.0,1.0,3.0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦..."
2,6931EFC26D229CCFCEA125D3F3C21E57,4.0,2.0,3.0,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 写 范例 三维 绘图 软件 枣 和 酸...
3,E780470C3BB0D340334BD08CDCC3C71A,2.0,2.0,4.0,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...
4,0D58894CD481D4B3039000661665C0DB,3.0,2.0,3.0,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅 饲育 指南 韩姨 贵阳 古琴 洛基 风速 英美 店主 好吃 重生 儿童 寻家记 锦里 ...,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...


In [15]:
df_all.shape

(188492, 8)

In [0]:
df_all = df_all.reset_index().drop(["index"],axis=1)

In [0]:
df_all.to_csv("drive/My Drive/sougou/data/all_data.csv")